In [1]:
# Processing and Displaying run results
import os
import pandas as pd

In [2]:
# Define a function to extract information from the file content
def extract_info_from_file(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    total_time = None
    final_accuracy = None
    for line in reversed(lines):
        if 'Total training time:' in line:
            total_time = float(line.strip().split(': ')[1].split(' ')[0])
        if 'accuracy:' in line:
            final_accuracy = float(line.strip().split(': ')[1].replace('%', ''))
            break  # Stop after the final accuracy is found as it is the first instance from the end

    return total_time, final_accuracy

# Function to parse file name
def parse_filename(file_name):
    parts = file_name.split('-')
    if len(parts) == 5:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3].split('.')[0]  # Remove file extension, prob need to add run number here
    else:
        optimizer, model, dataset, augmentation = None, None, None, None

    return optimizer, model, dataset, augmentation

In [3]:
# Specify the directory
folder_path = 'logs'  # Change 'folder_name' to the name of your folder

# DataFrame to store the results
results_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Total Time (s)', 'Final Accuracy (%)'])

results_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%)


In [4]:
# Process each file in the directory
for file in os.listdir(folder_path):
    if file.endswith('.out'):  # Check if the file is a .out file
        file_path = os.path.join(folder_path, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        optimizer, model, dataset, augmentation = parse_filename(file)
        
        # Append the results to the DataFrame
        results_df = results_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time
        }, ignore_index=True)

results_df['Total Time (min)'] = results_df['Total Time (s)'] / 60

results_df.head()

/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_42221/3471619082.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = results_df._append({


,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%),Total Time (min)
0,NaN,NaN,NaN,NaN,282.725515,8.87,4.712092


In [5]:
grouped_df = results_df.groupby(['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation']).agg({
    'Final Accuracy (%)': ['mean', 'std'],
    'Total Time (min)': ['mean', 'std']
}).reset_index()

# Rename the columns for clarity
grouped_df.columns = [
    'Optimizer', 
    'Model Architecture', 
    'Dataset', 
    'Data Augmentation', 
    'Final Accuracy Mean (%)', 
    'Final Accuracy Std (%)',
    'Total Time Mean (min)', 
    'Total Time Std (min)' 
]

grouped_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)


In [6]:
df_sorted = grouped_df.sort_values(by=['Dataset', 'Data Augmentation'])

df_sorted

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)


In [7]:
from tabulate import tabulate

In [8]:
'''
# Define the custom order for Data Augmentation
augmentation_order = ["basicaug", "cutout", "autoaug", "randaug"]
optimizer_order = ["sgd", "nonaccelgamsgd", "gamsgd"]

# Create a categorical type for Data Augmentation with the specified order
grouped_df["Data Augmentation"] = pd.Categorical(grouped_df["Data Augmentation"], categories=augmentation_order, ordered=True)

# Filter and sort DataFrame to match the desired format
df_sorted = grouped_df.sort_values(by=["Dataset", "Data Augmentation", "Optimizer"])
df_sorted = df_sorted.loc[df_sorted["Data Augmentation"] != "randaug"]
df_cifar10 = df_sorted[df_sorted["Dataset"] == "cifar10"]
df_cifar100 = df_sorted[df_sorted["Dataset"] == "cifar100"]

# Prepare the CIFAR-10 and CIFAR-100 parts of the table
table_cifar10 = df_cifar10.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()
table_cifar100 = df_cifar100.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()

table_cifar10 = table_cifar10[["Data Augmentation"] + optimizer_order]

# Add a new column to distinguish between CIFAR-10 and CIFAR-100
table_cifar10.insert(0, 'Dataset', 'CIFAR-10')
table_cifar100.insert(0, 'Dataset', 'CIFAR-100')

# Combine the tables
table = pd.concat([table_cifar10, table_cifar100], axis=0)

# Print the table
print(tabulate(table, headers='keys', tablefmt='pipe'))
'''

'\n# Define the custom order for Data Augmentation\naugmentation_order = ["basicaug", "cutout", "autoaug", "randaug"]\noptimizer_order = ["sgd", "nonaccelgamsgd", "gamsgd"]\n\n# Create a categorical type for Data Augmentation with the specified order\ngrouped_df["Data Augmentation"] = pd.Categorical(grouped_df["Data Augmentation"], categories=augmentation_order, ordered=True)\n\n# Filter and sort DataFrame to match the desired format\ndf_sorted = grouped_df.sort_values(by=["Dataset", "Data Augmentation", "Optimizer"])\ndf_sorted = df_sorted.loc[df_sorted["Data Augmentation"] != "randaug"]\ndf_cifar10 = df_sorted[df_sorted["Dataset"] == "cifar10"]\ndf_cifar100 = df_sorted[df_sorted["Dataset"] == "cifar100"]\n\n# Prepare the CIFAR-10 and CIFAR-100 parts of the table\ntable_cifar10 = df_cifar10.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()\ntable_cifar100 = df_cifar100.pivot(index="Data Augmentation", columns="Optimizer", values="Fin

### Gradient Norm Only Minimization (GNOM)

#### GNOM learning rate testing (20 epochs):

In [9]:
# Specify the directory
folder_path_gnom = 'logs/GNOM-Tests'  


In [10]:
def extract_epoch_time(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    epoch_time = 0
    count = 0
    for line in lines:
        if 'Epoch' and 'time:' in line:
            count += 1
            epoch_time += float(line.strip().split(': ')[1].split(' ')[0])
    return (epoch_time / count)

def extract_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    epochs = 20
    if len(parts) >= 6:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size = None, None, None, None, None, None
    
    if len(parts) == 7:
        learning_rate = parts[5]
        batch_size = parts[6].split('.')[0] 
        if batch_size == 'full':
            batch_size = 128
            epochs = 200
    else:
        learning_rate = parts[5][:-4]
        batch_size = 128
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

In [11]:
gnom_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom):
    if '-lr-' in file:  
        file_path = os.path.join(folder_path_gnom, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        epoch_time = extract_epoch_time(file_path)
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_test_info(file_path)
        
        gnom_df = gnom_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Learning Rate': float(learning_rate),
            'Batch Size': int(batch_size),
            'Epochs': epochs,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time,
            'Avg Epoch Time (s)': epoch_time
        }, ignore_index=True)

gnom_df['Total Time (min)'] = gnom_df['Total Time (s)'] / 60

gnom_df_sorted = gnom_df.sort_values(by=['Batch Size', 'Learning Rate'])

gnom_df_sorted

/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_42221/9759538.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_df = gnom_df._append({


,Optimizer,Model Architecture,Dataset,Data Augmentation,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Total Time (s),Avg Epoch Time (s),Total Time (min)
7,gnomsgd,resnet18,cifar10,basicaug,0.010,128,20,10.30,2390.612709,227.676276,39.843545
10,gnomsgd,resnet18,cifar10,basicaug,0.050,128,20,18.06,2398.976304,228.472647,39.982938
14,gnomsgd,resnet18,cifar10,basicaug,0.050,128,200,10.00,23864.758999,237.459076,397.745983
1,gnomsgd,resnet18,cifar10,basicaug,0.100,128,20,10.00,2394.460447,228.042485,39.907674
2,gnomsgd,resnet18,cifar10,basicaug,0.200,128,20,10.00,2384.751838,227.118328,39.745864
6,gnomsgd,resnet18,cifar10,basicaug,0.400,128,20,10.00,2393.539392,227.954680,39.892323
11,gnomsgd,resnet18,cifar10,basicaug,0.010,256,20,17.04,2241.874085,213.510471,37.364568
5,gnomsgd,resnet18,cifar10,basicaug,0.050,256,20,10.71,2237.419777,213.086812,37.290330
8,gnomsgd,resnet18,cifar10,basicaug,0.100,256,20,14.33,2247.514600,214.047492,37.458577
12,gnomsgd,resnet18,cifar10,basicaug,0.200,256,20,9.97,2252.608104,214.532993,37.543468


#### GNOM Gradient Norm and Hessian Eigenvalue testing:

In [12]:
def extract_ev_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    if len(parts) == 9:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
        epochs = parts[7]
    elif len(parts) == 8:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
        epochs = parts[7][:-4]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = None, None, None, None, None, None, None
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

def extract_grad_info(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    grad_norm = None
    top_ev = None
    smallest_ev = None
    for line in reversed(lines):
        if 'Smallest Hessian Eigenvalue:' in line:
            smallest_ev = float(line.strip().split(': ')[1])
        if 'Norm of the Gradient:' in line:
            grad_norm = float(line.strip().split(': ')[1])
        if 'Largest Hessian Eigenvalue:' in line:
            top_ev = float(line.strip().split(': ')[1])
            break

    return grad_norm, top_ev, smallest_ev

In [13]:
folder_path_gnom_ev = 'logs/GNOM-Saved-Models'  
gnom_ev_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)', 'Gradient Norm', 'Largest Eigenvalue of Hessian', 'Smallest Eigenvalue of Hessian'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_ev):
    file_path = os.path.join(folder_path_gnom_ev, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_ev_test_info(file_path)
    grad_norm, top_ev, smallest_ev = extract_grad_info(file_path)
    
    gnom_ev_df = gnom_ev_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev,
        'Smallest Eigenvalue of Hessian': smallest_ev
    }, ignore_index=True)

gnom_ev_df['Total Time (min)'] = gnom_ev_df['Total Time (s)'] / 60
gnom_ev_df['Avg Epoch Time (min)'] = gnom_ev_df['Avg Epoch Time (s)'] / 60
gnom_ev_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_ev_df_sorted = gnom_ev_df.sort_values(by=['Epochs','Optimizer','Batch Size'])

print("All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations")
gnom_ev_df_sorted


All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations


/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_42221/1462318495.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_ev_df = gnom_ev_df._append({


,Optimizer,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Gradient Norm,Largest Eigenvalue of Hessian,Smallest Eigenvalue of Hessian,Total Time (min),Avg Epoch Time (min)
5,gam,0.10,128,200,96.41,0.396285,45.6671,-2.5961,332.266464,3.306122
7,gnomsgd,0.05,128,200,10.00,0.034401,0.1001,-0.0973,396.257328,3.942846
0,gnomsgd,0.01,256,200,11.14,0.066849,0.5386,-0.1408,372.538592,3.706831
3,gnomsgd,0.01,512,200,16.97,0.111012,1.0676,-0.1032,354.544338,3.527792
4,nonaccelgam,0.10,128,200,95.59,0.273106,59.2035,-1.7100,786.008842,7.820976
11,sgd,0.10,128,200,95.50,0.057379,25.0349,-0.1720,83.996313,0.835771
10,gam,0.10,128,50,94.52,0.918708,32.4005,-2.5729,82.848856,3.248966
9,gnomsgd,0.05,128,50,10.00,0.062882,0.0998,-0.0332,99.930228,3.918811
2,gnomsgd,0.01,256,50,12.71,0.070527,0.1035,-0.0133,93.470016,3.665468
8,gnomsgd,0.01,512,50,14.24,0.158222,1.0770,-0.0801,88.931464,3.487495


### GNOM Noised

In [14]:
def extract_noise_info(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    batches_noised = None
    for line in reversed(lines):
        if 'Noise applied in' in line:
            batches_noised = float(line.strip().split(' ')[8])
            break

    return batches_noised

def extract_noised_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    if len(parts) >= 8:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
    
    if len(parts) == 10:
        epochs = parts[7]
        threshold = parts[8]
        radius = parts[9][:-4]
    else:
        epochs = parts[7][:-4]
        threshold, radius = 0.1, 0.01
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs, threshold, radius

In [18]:
folder_path_gnom_noised = 'logs/GNOM-Noised'  
gnom_noised_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)', 'Gradient Norm', 'Largest Eigenvalue of Hessian', 'Smallest Eigenvalue of Hessian'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_noised):
    file_path = os.path.join(folder_path_gnom_noised, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs, threshold, radius = extract_noised_test_info(file_path)
    grad_norm, top_ev, smallest_ev = extract_grad_info(file_path)
    batches_noised = extract_noise_info(file_path)
    
    gnom_noised_df = gnom_noised_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev,
        'Smallest Eigenvalue of Hessian': smallest_ev,
        'Batches Noise was Added (%)': batches_noised,
        'Noise Gradient Norm Threshold': threshold,
        'Noise Added Radius': radius
    }, ignore_index=True)

gnom_noised_df['Total Time (min)'] = gnom_noised_df['Total Time (s)'] / 60
gnom_noised_df['Avg Epoch Time (min)'] = gnom_noised_df['Avg Epoch Time (s)'] / 60
gnom_noised_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_noised_df_sorted = gnom_noised_df.sort_values(by=['Final Accuracy (%)'], ascending=False)

print("All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations")

gnom_noised_df_sorted

All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations


/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_42221/1017131250.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_noised_df = gnom_noised_df._append({


,Optimizer,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Gradient Norm,Largest Eigenvalue of Hessian,Smallest Eigenvalue of Hessian,Batches Noise was Added (%),Noise Gradient Norm Threshold,Noise Added Radius,Total Time (min),Avg Epoch Time (min)
7,gnomnoised,0.10,512,20,15.53,0.159442,2.5711,-3.7333,14.49,0.2,0.01,39.130230,3.726676
1,gnomnoised,0.01,256,20,14.93,0.168913,1.8545,-0.0385,47.96,0.5,0.1,55.096801,5.247302
5,gnomnoised,0.05,256,20,14.90,0.090542,0.9358,-0.0242,20.56,0.2,0.1,45.201966,4.304936
24,gnomnoised,0.05,512,20,14.76,0.446184,2.0480,-0.0727,0.00,0.1,0.01,35.989834,3.427591
23,gnomnoised,0.05,256,20,14.49,0.295037,1.1369,-0.3101,81.56,0.5,0.1,67.337877,6.413119
3,gnomnoised,0.05,128,20,13.75,0.123675,0.4434,-0.1482,30.51,0.1,0.01,62.348423,5.937923
8,gnomnoised,0.01,512,20,13.34,0.221421,0.2700,-0.0687,0.05,0.1,0.01,35.905507,3.419560
12,gnomnoised,0.01,256,20,13.31,0.201955,0.4627,-0.0580,20.20,0.2,0.1,45.209128,4.305617
0,gnomnoised,0.05,256,20,12.67,0.247105,1.1121,-0.0382,0.00,0.1,0.01,37.461890,3.567786
4,gnomnoised,0.01,128,20,12.62,0.629096,1.2759,-0.0431,0.00,0.1,0.5,40.107073,3.819707


### GNOM Normal vs. Noised

In [16]:
combined_results = pd.concat([gnom_ev_df_sorted, gnom_noised_df_sorted], ignore_index=True)
gnom_results = combined_results[combined_results['Optimizer'].isin(["gnomsgd"])]
gnom_results = gnom_results.sort_values(by=['Final Accuracy (%)'])
gnom_results

,Optimizer,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Gradient Norm,Largest Eigenvalue of Hessian,Smallest Eigenvalue of Hessian,Total Time (min),Avg Epoch Time (min),Batches Noise was Added (%),Noise Gradient Norm Threshold,Noise Added Radius
1,gnomsgd,0.05,128,200,10.00,0.034401,0.1001,-0.0973,396.257328,3.942846,NaN,NaN,NaN
7,gnomsgd,0.05,128,50,10.00,0.062882,0.0998,-0.0332,99.930228,3.918811,NaN,NaN,NaN
2,gnomsgd,0.01,256,200,11.14,0.066849,0.5386,-0.1408,372.538592,3.706831,NaN,NaN,NaN
8,gnomsgd,0.01,256,50,12.71,0.070527,0.1035,-0.0133,93.470016,3.665468,NaN,NaN,NaN
9,gnomsgd,0.01,512,50,14.24,0.158222,1.0770,-0.0801,88.931464,3.487495,NaN,NaN,NaN
3,gnomsgd,0.01,512,200,16.97,0.111012,1.0676,-0.1032,354.544338,3.527792,NaN,NaN,NaN


### GNOM Noised with Partitioned Gradient Calculation Dataset
#### Batch fixed at 256 and learning rate fixed at 0.01

In [21]:
folder_path_gnom_noised_part = 'logs/GNOM-Noised-Partitioned'  
gnom_noised_part_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)', 'Gradient Norm', 'Largest Eigenvalue of Hessian', 'Smallest Eigenvalue of Hessian'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_noised_part):
    file_path = os.path.join(folder_path_gnom_noised_part, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs, threshold, radius = extract_noised_test_info(file_path)
    grad_norm, top_ev, smallest_ev = extract_grad_info(file_path)
    batches_noised = extract_noise_info(file_path)
    
    gnom_noised_part_df = gnom_noised_part_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev,
        'Smallest Eigenvalue of Hessian': smallest_ev,
        'Batches Noise was Added (%)': batches_noised,
        'Noise Gradient Norm Threshold': threshold,
        'Noise Added Radius': radius
    }, ignore_index=True)

gnom_noised_part_df['Total Time (min)'] = gnom_noised_part_df['Total Time (s)'] / 60
gnom_noised_part_df['Avg Epoch Time (min)'] = gnom_noised_part_df['Avg Epoch Time (s)'] / 60
gnom_noised_part_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_noised_part_df_sorted = gnom_noised_part_df.sort_values(by=['Final Accuracy (%)'], ascending=False)

gnom_noised_part_df_sorted

/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_42221/4017986105.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_noised_part_df = gnom_noised_part_df._append({


,Optimizer,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Gradient Norm,Largest Eigenvalue of Hessian,Smallest Eigenvalue of Hessian,Batches Noise was Added (%),Noise Gradient Norm Threshold,Noise Added Radius,Total Time (min),Avg Epoch Time (min)
1,gnomnoised,0.01,256,test,10.64,1.656843,4.7026,-0.3062,0.0,0.5,0.1,9.280089,6.186704
0,gnomnoised,0.01,256,test2,9.94,3.340197,4.7062,-0.5637,0.0,0.5,0.1,4.742382,4.742367
